In [29]:
%matplotlib inline

In [2]:
import tensorflow as tf
import data_provider_raw
import compare_train
import models
import numpy as np
import matplotlib.pyplot as plt

from menpo.visualize import print_dynamic
slim = tf.contrib.slim

## Start a TF Session

In [3]:
nogpu_config = tf.ConfigProto(
    # Do not use a GPU device
    device_count = {'GPU': 1}
)

sess = tf.Session(config=nogpu_config)

## Setup the input tensor and the model

The dimensionality of the input for a single sample should be a multiple of 640 (40ms for a 16KHz singal).

In [4]:
wave = tf.placeholder(tf.float32, shape=(1, None, 640))

with slim.arg_scope([slim.batch_norm], is_training=False):
    prediction = models.get_model('audio')(wave, num_classes=2)

### Restore the weights of the model

In [22]:
variables_to_restore = slim.get_variables_to_restore()  
saver = tf.train.Saver(variables_to_restore)
model_path = slim.evaluation.tf_saver.get_checkpoint_state('ckpt/train/').model_checkpoint_path
saver.restore(sess, model_path)

print(model_path)

ckpt/train/model.ckpt-58696


In [23]:
from data_generator import read_wave
from pathlib import Path

### Feed the model the raw waveforms

In [28]:
preds = []
gts = []
names = []

data = data_provider_raw.get_split('addressee', 'devel')

for i, (audio, label, name) in enumerate(data):
    
    a = sess.run(prediction, feed_dict={wave: audio[None, :]})
    preds.append(a[0])
    names.append(name)
    gts.append(label.argmax())
    
    print_dynamic("{}, ac: {:.2f}%, rnd: {:.2f}%".format(
            i, np.mean(np.array(preds).argmax(1) == np.array(gts)) * 100, np.array(gts).mean()))
    
preds = np.array(preds)
gts = np.array(gts)

['cold' 'no_cold']
9595, ac: 33.28%, rnd: 0.89%                                                    

### Evaluate the model

In [27]:
from sklearn.metrics import confusion_matrix

b = confusion_matrix(gts, preds.argmax(1))
recalls = np.diag(b) / np.sum(b, axis=1, dtype=np.float)
np.mean(recalls)

0.61925070219642908